In [ ]:
from twython import Twython
import pandas as pd
import numpy as np
import datetime as dt
import time

In [ ]:
APP_KEY = 'OumzA854PAnWwA8VNjwZ6O1T1'
APP_SECRET = "aHWN7hVHFYGTJIqLG6d1sUgT2tzelmB7LNxtQjJz0RkmLDIFGW"

twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()

In [ ]:
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)

In [ ]:
locationsTemp=["chennai","delhi","india","ahmedabad","gujarat","tamil","jammu","srinagar","imphal","lucknow",
                "bengaluru","bangalore","bathinda","chandigarh","ludhiana","amritsar","punjab","arunachal","assam",
                "kolkata","westbengal","kerala","karnataka","andhra","telangana","hyderabad","madhya","uttar",
                "maharashtra","haryana",'Jammu','Srinagar','Delhi-NewDelhi','Bathinda','Dehradun','Chandigarh',
                'Ludhiana','Amritsar','Imphal','Lucknow','Jalandhar','Kolkata','Guwahati','Chennai','Patiala',
                'Bhubaneswar','Bengaluru','Patna','Jaipur','Coimbatore','Hyderabad','Ranchi','Thiruvananthapuram',
                'Shimla','Sangrur','Ahmedabad','Karnal','Pulwama','Puducherry','Gurgaon','Agartala','Madurai',
                'Tiruchirappalli','Bangkok','Salem']

locations = set()
for location in locationsTemp:
    locations.add(location.lower())
locations = pd.DataFrame(list(locations))
locations.columns = ["Locations"]
locations.to_csv("Locations of Interest.csv")

In [ ]:
keywords = ['protest','protests','riot','riots','violence','unrest','clash','bandh','issue','rally','demonstration','election','polls','attac']
keywords = pd.DataFrame(keywords)
keywords.columns = ["Keywords"]
keywords.to_csv("Keywords.csv")

In [ ]:
locations = pd.read_csv("Locations of Interest.csv", index_col=0)
keywords = pd.read_csv("Keywords.csv", index_col=0)

In [ ]:
keywordsQuery = ''.join(map(str, [keyword+" OR " for keyword in list(keywords['Keywords'])]))[:-4]
locationsQuery = ''.join(map(str, [location+" OR " for location in list(locations['Locations'])]))[:-4]

In [ ]:
locationQueries = []
locationsLength = 512 - len(keywordsQuery) - 1

if locationsLength > 0:
    
    while (locationsQuery[locationsLength] not in ['O','R',' ']):
        locationsLength=locationsLength-1
    locationsLength+=1
    locationQueries.append(locationsQuery[locationsLength:])
    locationsLength-=1
    while (locationsQuery[locationsLength] in ['O','R',' ']):
        locationsLength=locationsLength-1
    locationsLength+=1
    locationQueries.append(locationsQuery[:locationsLength])

In [ ]:
index = 1
date = dt.datetime.now()
for locationQuery in locationQueries:
    tweets = twitter.cursor(twitter.search, q=keywordsQuery + locationQuery + ' -filter:retweets', count=100, tweet_mode = "extended")
    tweetList = []
    cnt = 0
    for item in tweets:
        tweetList.append(item)
        cnt+=1
        if cnt > 40000:
            time.sleep(60*15+5)
            cnt = 0
    df = pd.DataFrame(tweetList);
    df.to_csv("tweets_collected_"+str(i)+"_"+str(date.day)+str(date.strftime("%m"))+".csv")
    index+= 1
    print(index)

In [ ]:
tweetsDF = pd.read_csv("Tweets3Weeks.csv")

In [ ]:
tweetsDF["userlocation"] = ""
tweetsDF["username"] = ""
tweetsDF["userscreen_name"] = ""
tweetsDF["userdescription"] = ""
tweetsDF["userfollowers_count"] = ""
tweetsDF["userfriends_count"] = ""
tweetsDF["userlisted_count"] = ""
tweetsDF["userfavourites_count"] = ""
tweetsDF["userverified"] = ""
tweetsDF["userstatuses_count"] = ""
tweetsDF["userfollowing"] = ""
tweetsDF["userfollow_request_sent"] = ""
tweetsDF["usercontributors_enabled"] = ""

In [ ]:
for index, row in tweetsDF.iterrows():
    if index%1000 == 0: 
        print("Progress: " + str(index/1000))
    tweetsDF.loc[index, "userlocation"] = row["user"]["location"]
    tweetsDF.loc[index, "username"] = row["user"]["name"]
    tweetsDF.loc[index, "userscreen_name"] = row["user"]["screen_name"]
    tweetsDF.loc[index, "userdescription"] = row["user"]["description"]
    tweetsDF.loc[index, "userfollowers_count"] = row["user"]["followers_count"]
    tweetsDF.loc[index, "userfriends_count"] = row["user"]["friends_count"]
    tweetsDF.loc[index, "userlisted_count"] = row["user"]["listed_count"]
    tweetsDF.loc[index, "userfavourites_count"] = row["user"]["favourites_count"]
    tweetsDF.loc[index, "userverified"] = row["user"]["verified"]
    tweetsDF.loc[index, "userstatuses_count"] = row["user"]["statuses_count"]
    tweetsDF.loc[index, "userfollowing"] = row["user"]["following"]
    tweetsDF.loc[index, "userfollow_request_sent"] = row["user"]["follow_request_sent"]
    tweetsDF.loc[index, "usercontributors_enabled"] = row["user"]["contributors_enabled"]

In [ ]:
tweetsDF["extractedlocation"] = ""
for index, row in tweetsDF.iterrows():
    if index%1000 == 0: 
        print("Progress: " + str(index/1000))
    sepr =  "" 
    for location in locations: 
        if (location.lower() in str(row["full_text"]).lower()):
            tweetsDF.loc[index, "extractedlocation"] = tweetsDF.loc[index, "extractedlocation"]+sepr+location
            sepr = ", "

In [ ]:
tweetsDf = "finallocation"
finalTweetsDf = tweetsDf.copy()
for index, row in tweetsDF.iterrows():
    if index%1000 == 0: 
        print("Progress: " + str(index/1000))
    for location in acledLocations: 
        tempLocation = location
        tempLocation2 = location
        if (tempLocation == "Delhi-New Delhi"):
            tempLocation = "delhi"
        elif (tempLocation == "Bengaluru"):
            tempLocation = "bangalore"
            tempLocation2 = "karnataka"
        elif (tempLocation == "Kolkata"):
            tempLocation = "west bengal"
        elif (tempLocation == "Guwahati"):
            tempLocation = "assam"
        elif (tempLocation in ("Lucknow", "Dehradun")):
            tempLocation = "uttar"
        elif (tempLocation in ("Sangrur", "Ludhiana", "Amritsar", "Chandigarh", "Bathinda", "Jalandhar", "Patiala")):
            tempLocation = "punjab"
        elif (tempLocation in ("Karnal", "Gurgaon", "Chandigarh"):
            tempLocation2 = "haryana"
        elif (tempLocation == "Pulwama"):
            tempLocation2 = "jammu"
        elif (tempLocation == "Ahmedabad"):
            tempLocation = "gujarat"
        elif (tempLocation in ("Chennai", "Coimbatore", "Madurai", "Tiruchirappalli", "Salem")):
            tempLocation = "tamil"
        elif (tempLocation in ("Imphal", "Agartala")):
            tempLocation = "arunachal"
        elif (tempLocation == "Hyderabad"):
            tempLocation = "andhra"
            tempLocation2 = "telangana"
        elif (tempLocation == "Thiruvananthapuram"):
            tempLocation = "kerala"
        if (tempLocation.lower() in row["extractedlocation"] or location.lower() in row["extractedlocation"] or tempLocation2 in row["extractedlocation"]):
            tweetsDF.loc[index, "finallocation"] = location
            finalTweetsDf = finalTweetsDf.append(pd.DataFrame(tweetsDF.loc[index:index]))

In [1]:
60*15+5

905